In [1]:
from prettyprinter import pprint

In [2]:
## add src folder to path
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

# print paths 
print("Current working directory:", os.getcwd())
print("Python executable:", sys.executable)
pprint(sys.version, width=80, indent=4)
print("Python path:", sys.path) 

Current working directory: d:\SYNEREPOS\rag-research-agent-fork\notebooks
Python executable: d:\SYNEREPOS\.venv\Scripts\python.exe
'3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 '
'bit (AMD64)]'
Python path: ['C:\\Users\\zohai\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', 'C:\\Users\\zohai\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'C:\\Users\\zohai\\AppData\\Local\\Programs\\Python\\Python313\\Lib', 'C:\\Users\\zohai\\AppData\\Local\\Programs\\Python\\Python313', 'd:\\SYNEREPOS\\.venv', '', 'd:\\SYNEREPOS\\.venv\\Lib\\site-packages', '__editable__.retrieval_graph-0.0.1.finder.__path_hook__', 'd:\\SYNEREPOS\\.venv\\Lib\\site-packages\\win32', 'd:\\SYNEREPOS\\.venv\\Lib\\site-packages\\win32\\lib', 'd:\\SYNEREPOS\\.venv\\Lib\\site-packages\\Pythonwin', 'd:\\SYNEREPOS\\rag-research-agent-fork\\src']


In [3]:
from langgraph.graph import END, START, StateGraph

In [4]:
from retrieval_graph.graph import graph

mygraph = graph 
print(mygraph.name)

RetrievalGraph


In [5]:
# from IPython.display import Image, display

# display(Image(mygraph.get_graph(xray=True).draw_mermaid_png()))


In [9]:
inputs = {
    "messages": [
        ("user", "What does Lilian Weng say about the types of agent memory?"),
    ]
}


Use langgraph server (optional)

In [7]:
# https://langchain-ai.github.io/langgraph/tutorials/langgraph-platform/local-server/#launch-langgraph-server

In [9]:
# from langgraph_sdk import get_client

# client = get_client(url="http://localhost:2024")

# async for chunk in client.runs.stream(
#     None,  # Threadless run
#     "agent", # Name of assistant. Defined in langgraph.json.
#     input={
#         "messages": [{
#             "role": "human",
#             "content": "What is LangGraph?",
#         }],
#     },
#     stream_mode="updates",
# ):
#     print(f"Receiving new event of type: {chunk.event}...")
#     print(chunk.data)
#     print("\n\n") 